# This notebook provides portfolio weights based on the historical data to the backtesting notebook

In [ ]:
# !pip uninstall deepdowmine
#!pip install git+https://github.com/dsman1823/deepdowmine.git
#!conda install pytorch torchvision torchaudio cudatoolkit=10.2 -c pytorch # <---- to Enable SVD 

In [2]:
import torch

import pandas as pd
import numpy as np

from datetime import datetime 
from deepdowmine.nn import LinearNetMine


In [3]:
RETS_FILE_PATH = 'historical_returns.csv'



loockback, gap, horizon = 50, 0, 5
n_assets = 5


## Load NN from dict

In [4]:
LINEAR_NET_OSHARPE_FILE_PATH = r'./NNs/linear_net_50x5_sharpe.pth' # LinearNetMine with ordinary Sharpe cost

In [45]:
fr'./NNs/linear_net_50x5_sharpe.pth_{1}'

'./NNs/linear_net_50x5_sharpe.pth_1'

In [54]:
# years parameter reprsenets amount of years after the backtest stasrt: the model dowloaded is the the model trained 
# on the period (train_start, back_test_start + years) 


# LinearNetMine with ordinary sharpe cost function
def load_linear_net_osharpe(years=0):
    print(years)
    network = LinearNetMine(1, loockback, n_assets, p=0.5)
    network.load_state_dict(torch.load(fr'./NNs/linear_net_50x5_sharpe_{years}.pth'))
    return network.eval()
    

## Core functions

In [32]:
def transform_rets_to_NN_input(rets):
    # tranform (loockback, n_assets) df into (1, 1, loockback, n_assets)=(sample_size, n_channels, loockback, n_assets) tensor 
    returns_np = rets.to_numpy()

    # Add the required dimensions: (n_samples, n_channels, lookback, n_assets)
    returns_np_expanded = np.expand_dims(returns_np, axis=0)  # Adds n_samples dimension
    returns_np_expanded = np.expand_dims(returns_np_expanded, axis=0)  
    return torch.from_numpy(returns_np_expanded).float()

def weights_from_NN(rets, network):
    X = transform_rets_to_NN_input(rets)
    return network(X).detach().numpy()[0]

# method is a function, used for obtaining the returns
def get_weights(method):
    rets = pd.read_csv(RETS_FILE_PATH, index_col=0)
    return method(rets)

## NNs and related methods


### NNs

networks[i] method is trained on (train_start, backtest_start + i years) data interval

In [55]:
years = [0, 1, 2]

linear_osharpe_networks = [load_linear_net_osharpe(years=i) for i in years]

0
1
2


### methods

In [52]:
linear_osharpe_methods = [lambda r: weights_from_NN(r, linear_osharpe_networks[i]) for i in years]

In [53]:
[linear_osharpe_methods[i](rets) for i in range(3)]

[array([ 0.02715866,  0.6021298 , -0.2664901 ,  0.5245884 ,  0.11261309],
       dtype=float32),
 array([ 0.02715866,  0.6021298 , -0.2664901 ,  0.5245884 ,  0.11261309],
       dtype=float32),
 array([ 0.02715866,  0.6021298 , -0.2664901 ,  0.5245884 ,  0.11261309],
       dtype=float32)]

# Server

In [44]:
RETRAINING_DATE1 = datetime(2021, 12, 15)

True

In [7]:
from flask import Flask, request, jsonify
from flask_cors import CORS 

import numpy as np

app = Flask(__name__)
CORS(app)  # Enable CORS for your Flask app


@app.route('/get_weights', methods=['GET'])
def get_weights():
    # Extract date string from the query parameters
    date_str = request.args.get('date', None)
    
    # Simple validation to check if date is provided
    if not date_str:
        return jsonify({"error": "Missing date parameter"}), 400

    # Try to convert the date string to a datetime object
    try:
        # Note that now we're only parsing the date, not the time
        date = datetime.strptime(date_str, "%Y-%m-%d").date()
    except ValueError:
        # If there is an error in parsing the date, return an error message
        return jsonify({"error": "Invalid date format. Please use YYYY-MM-DD format."}), 400

    
    if 
    

    # Return the vector as JSON, using the string representation of the date for simplicity
    return jsonify({"date": date_str, "vector": vector})


In [8]:
app.run(port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
